### 1. Import Library

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### 2.Crawl_data_q_a

#### 2.1.Find Elements

In [2]:
# Hàm để cào dữ liệu từ một trang
def crawl_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.find_all('div', class_='box-item-content')
    articles_data = []
    
    for _, article in enumerate(articles, start=1):
        # Lấy thông tin tác giả và thời gian
        author_info = article.find('div', class_='box-item-top')
        name = author_info.find('span', class_='author').get_text(strip=True)
        time = author_info.find('span', class_='time').get_text(strip=True)

        # Lấy tiêu đề câu hỏi
        question_element = article.find('a', class_='question-title')
        question = question_element.get_text(strip=True)
        
        # Lấy tình huống
        summary = article.find('div', class_='question-sapo').get_text(strip=True)
        
        # Lấy liên kết chi tiết
        detail_link = article.find('a', class_='box-viewmore')['href']
        detail_url = f"https://chinhsachonline.chinhphu.vn{detail_link}"
        
        # Gửi yêu cầu HTTP để lấy nội dung trang chi tiết
        detail_response = requests.get(detail_url)
        detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
        
        # Lấy câu trả lời từ trang chi tiết
        answer_div = detail_soup.find('div', class_='detail__rcontent')
        answer = answer_div.get_text(strip=True) if answer_div else 'No answer available'
        
        # Thêm thông tin bài viết vào danh sách
        articles_data.append({
            'Name': name,
            'Time': time,
            'Question': question,
            'Situation': summary,
            'Answer': answer
        })
    
    return articles_data


#### 2.2.Execute Function

In [3]:
articles_data = []

for page_num in range(1, 400):  # Thay đổi phạm vi để bao gồm số trang bạn cần
    page_url = f"https://chinhsachonline.chinhphu.vn/giao-thong-xay-dung-tai-nguyen-moi-truong/t1/13/trang-{page_num}.htm"
    articles_data.extend(crawl_page(page_url))
    

#### 2.3 Export csv

In [4]:
# Tạo DataFrame từ danh sách dữ liệu
df_laodong_tienthuong = pd.DataFrame(articles_data)
df_laodong_tienthuong.head(5)

,Name,Time,Question,Situation,Answer
0,Nguyễn Văn Ba,09:27 11/04/2025,Không sản xuất nông nghiệp có được gia hạn sử ...,"Tôi xin hỏi, cá nhân không trực tiếp sản xuất ...",Bộ Nông nghiệp và Môi trường trả lời vấn đề nà...
1,Trịnh Quang Nam,12:14 10/04/2025,Cho thuê phòng trọ có phải đăng ký môi trường?,"Tôi có 9 phòng cho thuê, tổng diện tích khoản...",Bộ Nông nghiệp và Môi trường trả lời vấn đề nà...
2,Lê Văn Thành,13:01 08/04/2025,Hộ kinh doanh có phải đăng ký môi trường?,Tôi đăng ký thành lập hộ kinh doanh từ năm 202...,Bộ Nông nghiệp và Môi trường trả lời vấn đề nà...
3,Văn Thuấn,14:11 06/04/2025,Căn cứ tính tổng hạn mức đất ở của hộ gia đình...,Tôi đề nghị cơ quan chức năng giải đáp về tổng...,Bộ Nông nghiệp và Môi trường trả lời vấn đề nà...
4,Võ Thị Thơ,08:23 04/04/2025,Viên chức có được nhận thừa kế đất lúa?,Em họ tôi là giáo viên mầm non trong biên chế ...,Bộ Nông nghiệp và Môi trường trả lời vấn đề nà...


In [5]:

# Lưu DataFrame vào file CSV
csv_file = '../Source_Fine-Tuning/data/giaothong_xaydung_tainguyen_moitruong_data.csv'
df_laodong_tienthuong.to_csv(csv_file, encoding='utf-8',index_label='Index')

print(f"Successfully saved data to {csv_file}")

Successfully saved data to ../Source_Fine-Tuning/data/giaothong_xaydung_tainguyen_moitruong_data.csv


In [6]:
df_laodong_tienthuong=pd.read_csv("../Source_Fine-Tuning/data/giaothong_xaydung_tainguyen_moitruong_data.csv",encoding='utf-8')

In [7]:
df_laodong_tienthuong.shape

(2849, 6)